## Read and parse XML
(find VUAMC.xml in the git folder and download it in one directory with this parser)

In [5]:
# import xml.etree.ElementTree as etree
from lxml import etree, objectify

root = etree.parse('VUAMC.xml')

## Cleanup xml schema/namespaces from tags ##    
for elem in root.getiterator():
    if not hasattr(elem.tag, 'find'): continue  # (1)
    i = elem.tag.find('}')
    if i >= 0:
        elem.tag = elem.tag[i+1:]
objectify.deannotate(root, cleanup_namespaces=True)

## Traverse XML tree and extract sentences containing metaphors

In [14]:
import pandas as pd

def extract_similes(root):
    rows = []
    for sent in root.findall('.//s'): # scan all sentences
        text = ''
        mflag = ''
        mrw = ''
        type_ = ''
        for word in sent.findall('.//w'): # for each word in sentence
            aseg = word.find('.//seg')
            if aseg is not None:
                if not aseg.text or not aseg.text.strip():
                    continue
                ft = aseg.text.strip()#.encode('UTF-8')
                if aseg.get('function') == 'mFlag': # flag for similes
                    mflag += ' ' + ft
                    text += ' ' + ft
                if aseg.get('function') == 'mrw':
                     # and not (not mflag): # start collecting keywords only after mflag
                    mrw += ' ' + ft
                    text += ' ' + ft
                    type_ = aseg.get('type')
            elif not (not word.text):
                text += ' ' + word.text.strip()#.encode('UTF-8')

        text = text.strip()
        mrw = mrw.strip()
        mflag = mflag.strip()
        if not (not mrw) and type_ == 'met': 
            rows.append([mflag, mrw, text])
    df = pd.DataFrame(rows)
    df.columns = ['_', 'mrw', 'sentence']
    return df
   

## This creates an xml file with metaphors and sentences w/met extracted from the corpus
(replace with your code, when you have a classifier)

In [15]:
df = extract_similes(root)
df.to_csv('metaphors.csv')